## Further reading

- [Is Fine-Tuning Still Valuable?](https://hamel.dev/blog/posts/fine_tuning_valuable.html)
- [When and Why to Fine Tune](https://www.youtube.com/watch?v=cPn0nHFsvFg)
- [From Prompt to Model: Fine-tuning when you've already deployed LLMs in prod](https://www.youtube.com/watch?v=4EPZZkVrXC4)
- [Deploying Fine-Tuned Models](https://www.youtube.com/watch?v=GzEcyBykkdo)
- [Fine Tuning OpenAI Models - Best Practices](https://www.youtube.com/watch?v=Q0GSZD0Na1s)
- [Fine-Tuning with Axolotl](https://www.youtube.com/watch?v=mmsa4wDsiy0)
- [Why fine-tuning is dead](https://www.youtube.com/watch?v=h1c_jmk97Ss)
- [Fine-Tuning Llama 3 and Using It Locally: A Step-by-Step Guide](https://www.datacamp.com/tutorial/llama3-fine-tuning-locally)
- [RAG vs Fine-Tuning: A Comprehensive Tutorial with Practical Examples](https://www.datacamp.com/tutorial/rag-vs-fine-tuning?utm_cid=19589720821&utm_aid=157098104375&utm_campaign=230119_1-ps-other~dsa-tofu~all_2-b2c_3-emea_4-prc_5-na_6-na_7-le_8-pdsh-go_9-nb-e_10-na_11-na&utm_loc=9212664-&utm_mtd=-c&utm_kw=&utm_source=google&utm_medium=paid_search&utm_content=ps-other~emea-en~dsa~tofu~tutorial~artificial-intelligence&gad_source=1&gad_campaignid=19589720821&gbraid=0AAAAADQ9WsFIxTj2FMxHfYtkNv25bmuXQ&gclid=Cj0KCQiA9t3KBhCQARIsAJOcR7wj9NfQPsOqKHX3h1x-Tiff_LxQP22G2oVC6YC5A8lgWagBU5tdXlEaAkmlEALw_wcB)
- [LoRA](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora)
- [Supervised fine-tuning - OpenAI](https://platform.openai.com/docs/guides/supervised-fine-tuning)
    - [Model optimization - OpenAI](https://platform.openai.com/docs/guides/model-optimization)
- [Using LangSmith to Support Fine-tuning](https://blog.langchain.com/using-langsmith-to-support-fine-tuning-of-open-source-llms/)
    - [Colab's notebook](https://colab.research.google.com/drive/1tpywvzwOS74YndNXhI8NUaEfPeqOc7ub?usp=sharing&ref=blog.langchain.com)